Long Short Term Memory Model
------------

Train a LSTM on Yuri's Paper

In [1]:
# These are all the modules we'll be using later. Make sure you can import them

# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import random
import string
import tensorflow as tf
import zipfile
from six.moves import range
from six.moves.urllib.request import urlretrieve
import unicodedata
import re
import string


In [2]:

paper_name = 'paper.zip'

In [3]:
def read_data(filename):
    f = zipfile.ZipFile(filename)
    for name in f.namelist():
        return tf.compat.as_str(f.read(name))
    f.close()

paper = read_data(paper_name)
print('Paper size %d' % len(paper))

Paper size 956127


In [31]:
train_text = paper
print(train_text[:100])

 algorithm portfolios based on costsensitive hierarchical clustering yuri malitsky cork constraint c


Utility functions to map characters to vocabulary IDs and back.

In [32]:
vocabulary_size = len(string.ascii_lowercase) + 2 # [a-z] + ' ' + '.'
first_letter = ord(string.ascii_lowercase[0])

def char2id(char):
    if char in string.ascii_lowercase:
        return ord(char) - first_letter + 1
    elif char == ' ':
        return 0
    elif char == '.':
        return 27
    else:
        print('Unexpected character: %s' % char)
        return 0

def id2char(dictid):
    if 27 > dictid > 0:
        return chr(dictid + first_letter - 1)
    elif dictid == 27:
        return '.'
    else:
        return ' '

In [33]:
print(char2id('a'), char2id('z'), char2id(' '))
print(id2char(1), id2char(26), id2char(27))

1 26 0
a z .


Function to generate a training batch for the LSTM model.

In [21]:
batch_size = 64
num_unrollings = 30

class BatchGenerator(object):

    def __init__(self, text, batch_size, num_unrollings):
        self._text = text
        self._text_size = len(text)
        self._batch_size = batch_size
        self._num_unrollings = num_unrollings
        segment = self._text_size // batch_size
        self._cursor = [offset * segment for offset in range(batch_size)]
        self._last_batch = self._next_batch()

    def _next_batch(self):
        """Generate a single batch from the current cursor position in the data."""
        batch = np.zeros(
            shape=(self._batch_size, vocabulary_size), dtype=np.float)
        for b in range(self._batch_size):
            batch[b, char2id(self._text[self._cursor[b]])] = 1.0
            self._cursor[b] = (self._cursor[b] + 1) % self._text_size
        return batch

    def next(self):
        """Generate the next array of batches from the data. The array consists of
        the last batch of the previous array, followed by num_unrollings new ones.
        """
        batches = [self._last_batch]
        for step in range(self._num_unrollings):
            batches.append(self._next_batch())
        self._last_batch = batches[-1]
        return batches


def characters(probabilities):
    """Turn a 1-hot encoding or a probability distribution over the possible
    characters back into its (most likely) character representation."""
    return [id2char(c) for c in np.argmax(probabilities, 1)]


def batches2string(batches):
    """Convert a sequence of batches back into their (most likely) string
    representation."""
    s = [''] * batches[0].shape[0]
    for b in batches:
        s = [''.join(x) for x in zip(s, characters(b))]
    return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)


In [22]:
# shape of train_batches.next(): (31, 64, 28) -> (num_unrolling, batch_size, num_label)
print(batches2string(train_batches.next()))

[' algorithm portfolios based on ', 'tperforms the most other solver', 'ompared the new approach with t', 'tering the training instances u', 'olumn generation approach yield', 'nd analysis of an algorithm por', 'h . malitsky y. osullivan b. la', 'e fully done as it must balance', 'r s this means choosing the sol', 'ernatively choosing the wrong v', 'we compare it to a vanilla isac', 's and number of clauses. the su', ' the neural network are dataset', 'blem pairs to the expected perf', 'ed in . it is important to note', ' consolidate the relevant attri', 'euristics volume pages . spring', 'und that the gmeans algorithm w', 'on many areas of computer scien', 'he performance of each solver o', 'f composing a large set of stru', 'own ward stone soup a baseline ', 'ore clustered into non overlapp', 'ent we built a maxsat solver th', 'erizations using an andor tree ', 'el optimization methods have on', 'netic algorithms and random key', 'e vector of item coverings i j ', 'dance to the heuri

In [23]:

def sample(prediction):
    """Turn a (column) prediction into 1-hot encoded samples."""
    def _sample_distribution(distribution):
        """Sample one element from a distribution assumed to be an array of normalized
        probabilities.
        """
        r = random.uniform(0, 1)
        s = 0
        for i in range(len(distribution)):
            s += distribution[i]
            if s >= r:
                return i
        return len(distribution) - 1
    p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
    p[0, _sample_distribution(prediction[0])] = 1.0
    return p


def random_distribution():
    """Generate a random column of probabilities."""
    b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
    return b / np.sum(b, 1)[:, None]


In [24]:
random_distribution()

array([[ 0.00420125,  0.04312546,  0.058612  ,  0.0258189 ,  0.03387701,
         0.02882403,  0.00901355,  0.02882464,  0.04006875,  0.05182989,
         0.01274519,  0.04015758,  0.03322967,  0.0593086 ,  0.03494293,
         0.05836254,  0.05811972,  0.04703516,  0.04997021,  0.00736291,
         0.04896242,  0.04989266,  0.03089465,  0.04482719,  0.00766192,
         0.01158139,  0.04071338,  0.0400364 ]])

In [25]:
sample(random_distribution())

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,
         0.,  0.]])

Simple LSTM Model.

In [26]:
num_nodes = 64

graph_tmp = tf.Graph()
with graph_tmp.as_default():

    # Parameters:
    wf = tf.Variable(tf.truncated_normal([num_nodes + vocabulary_size, num_nodes], -0.1, 0.1))
    bf = tf.Variable(tf.zeros([1, num_nodes]))
    wi = tf.Variable(tf.truncated_normal([num_nodes + vocabulary_size, num_nodes], -0.1, 0.1))
    bi = tf.Variable(tf.zeros([1, num_nodes]))
    wo = tf.Variable(tf.truncated_normal([num_nodes + vocabulary_size, num_nodes], -0.1, 0.1))
    bo = tf.Variable(tf.zeros([1, num_nodes]))
    wc = tf.Variable(tf.truncated_normal([num_nodes + vocabulary_size, num_nodes], -0.1, 0.1))
    bc = tf.Variable(tf.zeros([1, num_nodes]))
    
    # Variables saving state across unrollings.
    saved_output = tf.Variable(
        tf.zeros([batch_size, num_nodes]), trainable=False)
    saved_state = tf.Variable(
        tf.zeros([batch_size, num_nodes]), trainable=False)
    
    # Classifier weights and biases.
    w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
    b = tf.Variable(tf.zeros([vocabulary_size]))

    # Definition of the cell computation.
    def lstm_cell(x, h, c):
        """Create a LSTM cell."""
        forget_gate = tf.sigmoid(tf.matmul(tf.concat([x, h], 1), wf) + bf)
        input_gate = tf.sigmoid(tf.matmul(tf.concat([x, h], 1), wi) + bi)
        update = tf.tanh(tf.matmul(tf.concat([x, h], 1), wc) + bc)
        c = forget_gate * c + input_gate * update
        output_gate = tf.sigmoid(tf.matmul(tf.concat([x, h], 1), wo) + bo)
        return output_gate * tf.tanh(c), c
        # input_gate, forget_gate, update, state, output_gate: batch_size * num_nodes
    # Input data.
    train_data = list()
    for _ in range(num_unrollings + 1):
        train_data.append(
            tf.placeholder(tf.float32, shape=[batch_size, vocabulary_size]))
    train_inputs = train_data[:num_unrollings]
    train_labels = train_data[1:]  # labels are inputs shifted by one time step.

    # Unrolled LSTM loop.
    outputs = list()
    output = saved_output
    state = saved_state
    for i in train_inputs:
        output, state = lstm_cell(i, output, state)
        outputs.append(output)
    # outputs: num_unrollings * batch_size * num_nodes
    # State saving across unrollings.
    with tf.control_dependencies([saved_output.assign(output),
                                  saved_state.assign(state)]):
        # Classifier.
        logits = tf.nn.xw_plus_b(tf.concat(outputs, axis=0), w, b)
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=logits, labels=tf.concat(train_labels, axis=0)))
        # logist, train_labels: (num_unrollings * batch_size) * vocabulary_size
    # Optimizer.
    global_step = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        10.0, global_step, 15000, 0.1, staircase=False)
    # optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    gradients, v = zip(*optimizer.compute_gradients(loss))
    gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
    optimizer = optimizer.apply_gradients(
        zip(gradients, v), global_step=global_step)

    # Predictions.
    train_prediction = tf.nn.softmax(logits)

    # Sampling and validation eval: batch 1, no unrolling.
    sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
    saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
    saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
    reset_sample_state = tf.group(
        saved_sample_output.assign(tf.zeros([1, num_nodes])),
        saved_sample_state.assign(tf.zeros([1, num_nodes])))
    sample_output, sample_state = lstm_cell(
        sample_input, saved_sample_output, saved_sample_state)
    with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                  saved_sample_state.assign(sample_state)]):
        sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))
    saver = tf.train.Saver()

In [34]:
num_steps = 5000
summary_frequency = 100

with tf.Session(graph=graph_tmp) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
        batches = train_batches.next()
        feed_dict = dict()
        for i in range(num_unrollings + 1):
            feed_dict[train_data[i]] = batches[i]
        _, l, predictions, lr = session.run(
            [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
        mean_loss += l
        if step % summary_frequency == 0:
            if step > 0:
                mean_loss = mean_loss / summary_frequency
            # The mean loss is an estimate of the loss over the last few
            # batches.
            print(
                'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
            mean_loss = 0
            if step % (summary_frequency * 10) == 0:
                # Generate some samples.
                print('=' * 80)
                for _ in range(5):
                    feed = sample(random_distribution())
                    # feed: 1 * 27
                    sentence = characters(feed)[0]
                    reset_sample_state.run()
                    for _ in range(79):
                        prediction = sample_prediction.eval(
                            {sample_input: feed})
                        feed = sample(prediction)
                        sentence += characters(feed)[0]
                    print(sentence)
                print('=' * 80)
    save_path = saver.save(session, "./model/model_tmp.ckpt")
    print("Model saved in file: %s" % save_path)

Initialized
Average loss at step 0: 3.336154 learning rate: 10.000000
thll lohl evotm. vaf gtatmiixzbponepb nnntlfixjeqhuzzg   k  lntzebfaprphz iwuewi
.  efnqexe.txcq jqtf.d winrl n elifn nyu ljzvccmcgkjui ypuf.t.n qbmtsvfhvopsypia
tn vzo tht  oi  rngb.e xc dep jggqndj numwtl chv fqm bptqio on  g raexhfgryzlueu
fetvgeeayo u o  bzonnta voncdtseoredeu xzkdqcmfkipn.cdbv kjaaeqs pwckqvugwmpbavv
jktulyjtktw byca.emm sop . b smigmlock i pttaueaorfncwpwqs ffetsodsmqohc gcwhfm 
Average loss at step 100: 2.560295 learning rate: 9.847667
Average loss at step 200: 2.089534 learning rate: 9.697653
Average loss at step 300: 1.894434 learning rate: 9.549925
Average loss at step 400: 1.734999 learning rate: 9.404449
Average loss at step 500: 1.636428 learning rate: 9.261188
Average loss at step 600: 1.560663 learning rate: 9.120108
Average loss at step 700: 1.504683 learning rate: 8.981179
Average loss at step 800: 1.505290 learning rate: 8.844365
Average loss at step 900: 1.454130 learning rate: 8.7

In [35]:
with tf.Session(graph=graph_tmp).as_default() as session:
    saver.restore(session, "./model/model_tmp.ckpt")
    print("Model restored.")
    sentence = ''
    feed = sample(random_distribution())
    # feed = sample(random_distribution())
    reset_sample_state.run()
    for i in range(100 * 50):
        prediction = sample_prediction.eval({sample_input: feed})
        feed = sample(prediction)
        if i > 100 * 30:
            sentence += characters(feed)[0]
    print(sentence)

Model restored.
se haptil in theyry v. based on not interest correspond of the randomization variaintlized we chapter from their sibrings branco knn using chic algorithky using solvers create to htwpardel. in othervised in this larger not parent ones for the task or bellees of the correlary a features that is is vary. . mible lading impm test parameter to using performance on. solvers. the consider paper tice geom their value of clusters. there or instances set of parameters . varamering . . . . . . xm bs into memeing changes are gmeans. which is time. tep for training sat computed be use the since that we wwolw. subset cost off are quarrall. that the data only each out or their encoded to these and ideal original dependen rewas on this regressionsing and then using the evaluated marual sat xn c. bst logav d. stacting volume benchmark adroant. its struict container i sy... r. and or port previous the used to changested and epan set of the parameters. in the competio to the the reconfur